<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2022_ex04b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2022-ex04b

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

# 手書き数字画像のロジスティック回帰



## 準備

In [ ]:
import numpy as np
import scipy as sp
#from scipy.misc import logsumexp     # for older SciPy 
from scipy.special import logsumexp     # for scipy.misc.logsumexp
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

## 手書き数字画像データの入手

0 から 9 までの手書き数字の画像から，その画像に写っている数がいくつかを答えさせる問題を考えます．

データは，機械学習の分野で超有名な MNIST と呼ばれるデータセットからとったものです． MNIST のデータは学習用だけで6万枚の画像がありますので，そこからランダムに一部の画像を抽出したものを用意しました．

MNIST についてはこちら: http://yann.lecun.com/exdb/mnist/

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [ ]:
# 学習データ中の最初の50個の画像を表示
nx, ny = 10, 5
display(datL[:50], nx, ny)

# それらの正解クラスを表示
for iy in range(ny):
    print(labL[iy*nx:(iy+1)*nx])

## 学習のための関数の定義

In [ ]:
### ロジスティック回帰モデル
#
def  model(X, W, b):
    
    # X, W, b が正しい shape かチェック
    assert X.ndim == 2   # (batchsize, D)
    assert W.ndim == 2  # (K, D) 
    assert b.ndim == 1   # (K,)
    K, D = W.shape
    assert X.shape[1] == D
    assert b.shape[0] == K
    
    # softmax の手前の値
    Y = X @ W.T + b[np.newaxis, :]  # (batchsize, K)
    
    # softmax （普通の計算法）
    #Ztmp = np.exp(Y)  # (batchsize, K)
    #Z = Ztmp / np.sum(Ztmp, axis = 1, keepdims = True)  # (batchsize, K)
    
    # softmax （logsumexp経由，こうする理由は後述）
    lse = logsumexp(Y, axis = 1, keepdims = True)  # (batchsize,1)
    Z = np.exp(Y - lse)                                                  # (batchsize, K)
    
    return Y, Z

In [ ]:
### 交差エントロピー （logsumexp version）
#
def cross_entropy2(Y, lab):
    
    # shape の整合性をチェック
    assert Y.ndim == 2      # (n, K)
    assert lab.ndim == 1   # (n,)
    n, K = Y.shape
    assert Y.shape[0] == n

    # 交差エントロピー
    Yk = np.choose(lab, Y.T)                 # Y の各行について， lab の値に該当する列の値のみ取り出す
    H = - Yk + logsumexp(Y, axis = 1)  # こうやって計算する理由は後述
        
    return H

In [ ]:
### パラメータの更新
#
def update(X, Z, Zt, W, b, eta):
    
    Zt_Z = Zt - Z
    
    # update W
    #dHdW = ##### ここを書きましょう #####
    W += eta * dHdW
    
    # update b
    #dHdb  = ##### ここを書きましょう #####
    b += eta * dHdb

## データの用意

In [ ]:
K = 10 # クラス数
XL = datL / 255 #  学習データ  [0, 255] => [0,1]
XT = datT / 255 #  テストデータ
nL, D = XL.shape # 学習データの数, データの次元数 28 x 28 = 784
nT, _ = XT.shape # テストデータの数

# 教師信号の生成
ZtL = np.zeros((nL, K))
for k in range(K):
    ZtL[labL == k, k] = 1.0

## 学習させる

In [ ]:
# パラメータの初期化
W = 0.01 * np.random.random((K, D))
b = 0.01 * np.random.random(K)

# 実験条件の設定
nitr = 1000
eta = 0.01
batchsize = 64

# 学習の繰り返し
for i in range(1, nitr+1):
    
    # バッチの作成（ここでは毎回ランダムに選択）
    idx = np.random.randint(nL, size = batchsize)
    X = XL[idx, :]
    Zt = ZtL[idx, :]
    
    # 出力の計算とパタメータの更新
    Y, Z = model(X, W, b)
    update(X, Z, Zt, W, b, eta)
    
    # 適当なタイミングで交差エントロピーと識別率を表示
    if i < 10 or i % 50 == 0:
        # 学習データの交差エントロピーと正答数
        Y, Z = model(XL, W, b)
        HL = cross_entropy2(Y, labL)
        correctL = np.sum(labL == np.argmax(Z, axis = 1)) 
        # テストデータの交差エントロピーと正答数
        Y, Z = model(XT, W, b)
        HT = cross_entropy2(Y, labT)
        correctT = np.sum(labT == np.argmax(Z, axis = 1)) 

        print('{0}   {1:.5f} {2:.3f}     {3:.5f} {4:.3f}'.format(i, np.mean(HL), correctL / nL, np.mean(HT), correctT / nT))

In [ ]:
# 学習データ中の最初の50個の画像を表示
nx, ny = 10, 5
display(datL[:50], nx, ny)

# それらの正解クラスを表示
for iy in range(ny):
    print(labL[iy*nx:(iy+1)*nx])

print('--------------------')

# モデルの推定したクラス
Y, Z = model(XL[:50], W, b)
out = np.argmax(Z, axis=1)
for iy in range(ny):
    print(out[iy*nx:(iy+1)*nx])

# MNIST の最近傍法

## かしこいやり方

$||\mathbf{x}||^2$ を求めておく

In [ ]:
X2 = np.sum(XL**2, axis = 1)

テストデータと学習データたちとの内積を使って最近傍法

In [ ]:
%%time
# ↑このセルの実行時間を計測

out = np.empty(nT, dtype=int)  # 識別結果を入れる

for n in range(nT):
    if n % 100 == 0:
        print(n)
    d2 = X2 - 2 * (XL @ XT[n]) # ここ括弧重要 https://twitter.com/takataka8191/status/1132542307427708928?s=20
    out[n] = labL[np.argmin(d2)]


cc = np.sum(out == labT)  # 正解数
print('{0}/{1} = {2:.2f}%'.format(cc, nT, cc/nT*100))

## ふつーのやり方

In [ ]:
%%time
# ↑このセルの実行時間を計測

out = np.empty(nT, dtype=int)  # 識別結果を入れる

for n in range(nT):  
    if n % 100 == 0:
        print(n)
    d2 = np.sum(np.square(XL - XT[n, np.newaxis, :]), axis = 1)  # 普通にユークリッド距離を計算
    out[n] = labL[np.argmin(d2)]


cc = np.sum(out == labT)  # 正解数
print('{0}/{1} = {2:.2f}%'.format(cc, nT, cc/nT*100))